In [73]:
import nltk
import requests
import re
import unicodedata
import kagglehub
from collections import defaultdict
import pandas
from nltk.corpus import stopwords
from bs4 import BeautifulSoup


from nltk.tokenize import SpaceTokenizer, TabTokenizer, word_tokenize

In [52]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pende\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pende\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pende\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Pende\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Pende\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [53]:
en_contractions_path = kagglehub.dataset_download('ishivinal/contractions')
fr_contractions_path = kagglehub.dataset_download('johnpendenque/french-abbreviations')

In [54]:
response = requests.get('https://fr.wikipedia.org/wiki/Selena_Gomez')

In [55]:
soup = BeautifulSoup(response.content, 'html.parser')

In [82]:
# body = soup.find('body')
body = soup.find('main', attrs={'id': 'content'})
body_content = body.find('div', attrs={'id': 'bodyContent'})

In [83]:
text = body_content.text.lower()

## Handle abbreviations

In [84]:
import pathlib

In [85]:
fullpath = pathlib.Path(fr_contractions_path).joinpath('abreviations.csv')

df = pandas.read_csv(fullpath, sep=';')

In [86]:
for item in df.itertuples():
    value = df.loc[item.Index, 'abbreviation']
    fullform = df.loc[item.Index, 'meaning']
    text = re.sub(r'\b' + value + r'\b', fullform, text)

## Normalize text

In [87]:
# Remove special formatting and markup
# Remove references like (en), (d), etc.
clean_text = re.sub(r'\([^)]*\)', '', text)

# Remove square brackets and their contents
clean_text = re.sub(r'\[[^]]*\]', '', clean_text)

### Extract metadata

Extract some general meta data content from the text before complete pre processing

In [88]:
# TODO: Needs to be improved 

metadata_patterns = [
    # Pattern: Key: Value or Key - Value
    (r'([A-Za-z]+[A-Za-z\s]+)[\:\-]\s*([^,\n]+)', 'general'),
    # Name/title pattern
    (r'Nom(?:\sde\s\w+)?\s*[\:\-]?\s*([^\n,]+)', 'name'),
    # Date pattern
    (r'(?:Naissance|Date)[\:\-]?\s*(\d{1,2}(?:\s+\w+|\-\d{1,2})\s+\d{4})', 'birth_date'),
    # Age pattern
    (r'(?:\(\s*(\d+)\s*ans\s*\))', 'age'),
    # Location pattern
    (r'(?:Domicile|Lieu)[\:\-]?\s*([^\n,]+)', 'location'),
    # Profession pattern
    (r'(?:Activité|Profession|Métier)[\:\-]?\s*([^\n,]+)', 'profession'),
]

document = {
    'metadata': {},
    'sections': [],
    'structured_content': defaultdict(list)
}

for pattern, key in metadata_patterns:
    matches = re.findall(pattern, clean_text, re.IGNORECASE)
    if matches:
        if key == 'general':
            for match in matches:
                if isinstance(match, tuple) and len(match) >=2:
                    k, v = match[0].strip(), match[1].strip()
                    document['metadata'][k.lower()] = v
        else:
            document['metadata'][key] = matches[0].strip()


In [89]:
section_patterns = [
    r'([A-Z][A-Z\s]+)[\:\.\n]',  # ALL CAPS followed by colon or period
    r'^([A-Z][a-z]+)[\:\.]',     # Capitalized word at start of line
    r'(\w+)[\:]$'                # Word followed by colon at end of line
]

current_section = 'main'
lines = clean_text.split('\n')

for i, line in enumerate(lines):
    # Check if this line is a section header
    is_header = False
    for pattern in section_patterns:
        if re.search(pattern, line):
            # Extract section name
            section_name = re.search(pattern, line).group(1).strip().lower()
            current_section = section_name
            document['sections'].append(section_name)
            is_header = True
            break
    
    # If not a header, add content to current section
    if not is_header and line.strip():
        document['structured_content'][current_section].append(line.strip())

document

{'metadata': {'auteure': 'compositrice-interprète',
  'ologie\nmouvement pro': 'choix',
  'dance': 'par procuration196]',
  'o pop muchmusic video award for peoples choice': "favourite international video  american music award de l'artiste féminine préférée - pop / rock los premios 40 principales for best international video  leading ladies of entertainment  prix d'interprétation féminine du festival de cannes chevalier des article et des lettres 15e cérémonie des teen choice awardsteen choice awards 201016e cérémonie des teen choice awards13e cérémonie des teen choice awards14e cérémonie des teen choice awards11e cérémonie des teen choice awardsfilms notables",
  'les sorciers de waverly placesignaturemodifier': 'modifier le code - modifier wikidata',
  'est une auteure': 'compositrice-interprète',
  'monte': 'carlo',
  'celui': 'contraire en droit canadien. sort trois albums. le groupe siècle séparent finalement en 2012. en 2013',
  'en espagnol': 'revelación',
  'seaux sociaux': 'qu

In [62]:
with open('./raw_text.txt', mode='w', encoding='utf-8') as f:
    f.write(clean_text)

In [ ]:
# Handle unicode characters and accents
clean_text = unicodedata.normalize('NFKD', text).encode(
    'ASCII', 'ignore').decode('utf-8')

# Remove numbers and punctuation
clean_text = re.sub(r'[^\w\s]', ' ', clean_text)

# Remove extra whitespace, newlines, tabs
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

In [13]:
tokens = word_tokenize(clean_text, language='french')

In [ ]:
french_stop_words = set(stopwords.words('french'))
english_stop_words  = set(stopwords.words('english'))

stop_words = french_stop_words.union(english_stop_words)
tokens = [token for token in tokens if token not in stop_words]

## Joint text and print

In [15]:
text = ' '.join(tokens)

In [16]:
with open('./test.txt', mode='w', encoding='utf-8') as f:
    f.write(text)

In [23]:
import datetime

In [34]:
result = re.search(r'naissance\s(\d+\s\w+\s\d+)', text)
if result:
    date_of_birth = result.group(1)
    months = {
        'janvier': 1,
        'fevrier': 2,
        'mars': 3,
        'avril': 4,
        'mai': 5,
        'juin': 6,
        'juillet': 7,
        'aout': 8,
        'septembre': 9,
        'octobre': 10,
        'novembre': 11,
        'decembre': 12
    }

    day, month, year = date_of_birth.split(' ')
    month = months[month]

    date = f'{day} {month} {year}'
    d = datetime.datetime.strptime(date, '%d %M %Y').date()
    print(d)

1989-01-13
